# Still in a rough state, need cleaner structures and encapsulations.

/scraper/src/cvh_scraper.py

In [ ]:
!pip install selenium webdriver_manager -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def scrape_data_collection_ids_for_offset(base_url, offset):
    """
    Scrapes data collection IDs from a given page based on the offset.

    Args:
    - base_url (str): The base URL to scrape, excluding the offset query parameter.
    - offset (int): The offset to apply to the base URL for pagination.

    Returns:
    - List of data collection IDs for the given offset (list).
    """
    # Set up Chrome options for headless execution
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Set up Selenium WebDriver with headless Chrome
    driver = webdriver.Chrome(options=chrome_options)

    data_collection_ids = []

    # Construct the URL with the provided offset
    url = f"{base_url}{offset}"

    # Open the URL
    driver.get(url)

    # Wait for the dynamic content to load
    time.sleep(5)  # Adjust the sleep time if necessary

    # Find all elements with the specified class
    rows = driver.find_elements(By.CLASS_NAME, 'spms-row')

    # Extract data-collection-id from each row
    for row in rows:
        data_collection_ids.append(row.get_attribute('data-collection-id'))

    # Close the WebDriver
    driver.quit()

    return data_collection_ids

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import concurrent
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import pandas as pd
import time
import os

def setup_driver(headless=True):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
    # service = Service(ChromeDriverManager().install())
    # driver = webdriver.Chrome(service=service, options=chrome_options)

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def wait_for_element(driver, by_method, value, timeout=10, retry_interval=3, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by_method, value)))
        except TimeoutException:
            # print(f"Retry {retries+1}/{max_retries} for element {value} after timeout.")
            time.sleep(retry_interval)
            retries += 1
        finally:
            # print(f"Retry {retries+1}/{max_retries} for element {value} after timeout.")
            time.sleep(retry_interval)
    # raise TimeoutException(f"Element {value} not found after {max_retries} retries.")
    return "Missing Value or Timeout"  # Return None if element is not found

def fetch_data(collection_id):
    driver = setup_driver()
    try:
        print(f"Fetching data for collection: {collection_id}\n.")
        driver.get(f"https://www.cvh.ac.cn/spms/detail.php?id={collection_id}")

        # Fetch specific details as per your logic
        data = {
            'Collection ID': collection_id,
            'Image Link': wait_for_element(driver, By.ID, "spm_image", max_retries=5).get_attribute('src'),
            'Phylum (门)': wait_for_element(driver, By.ID, "taxon_phy_c", max_retries=5).text,
            'Order (目)': wait_for_element(driver, By.ID, "taxon_ord_c", max_retries=5).text,
            'Family (科)': wait_for_element(driver, By.ID, "taxon_fam_c", max_retries=5).text,
            'Genus (属)': wait_for_element(driver, By.ID, "taxon_gen_c", max_retries=5).text,
            'Scientific Name': wait_for_element(driver, By.ID, "formattedName", max_retries=5).text,
            'Chinese Name': wait_for_element(driver, By.ID, "chineseName", max_retries=5).text,
            'Identified By': wait_for_element(driver, By.ID, "identifiedBy", max_retries=5).text,
            'Date Identified': wait_for_element(driver, By.ID, "dateIdentified", max_retries=5).text,
            'Collector': wait_for_element(driver, By.ID, "recordedBy", max_retries=5).text,
            'Collection Number': wait_for_element(driver, By.ID, "recordNumber", max_retries=5).text,
            'Collection Date': wait_for_element(driver, By.ID, "verbatimEventDate", max_retries=5).text,
            'Collection Location': wait_for_element(driver, By.ID, "locality", max_retries=5).text,
            'Altitude': wait_for_element(driver, By.ID, "elevation", max_retries=5).text,
            'Habitat': wait_for_element(driver, By.ID, "habitat", max_retries=5).text,
            'Phenology': wait_for_element(driver, By.ID, "reproductiveCondition", max_retries=5).text
        }
        return data
    finally:
        driver.quit()

def fetch_collection_data_concurrently(path, collection_ids, max_workers=10):
    results = []

    # Initialize an empty DataFrame to hold column names. This is useful if you expect your first few calls might fail and return None.
    name = next((cid for cid in collection_ids if cid is not None), "default")

    # Concatenate path and name to form temp_path
    temp_path = f"{path}/{name}.csv"
    print(temp_path)

    # Ensure the directory exists before attempting to write to it
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)

    pd.DataFrame(columns=['Collection ID', 'Image Link', 'Phylum (门)', 'Order (目)', 'Family (科)',
                          'Genus (属)', 'Scientific Name', 'Chinese Name', 'Identified By',
                          'Date Identified', 'Collector', 'Collection Number', 'Collection Date',
                          'Collection Location', 'Altitude', 'Habitat', 'Phenology']).to_csv(temp_path, mode='w', index=False, encoding='utf-8-sig')

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for cid in collection_ids:
            try:
                # Submitting the fetch_data function to the executor
                future = executor.submit(fetch_data, cid)
                futures.append(future)
            except Exception as submit_error:
                print(f"Failed to submit task for {cid}: {submit_error}")

        results = []
        for future in tqdm(as_completed(futures), total=len(collection_ids)):
            try:
                result = future.result()  # This will raise an exception if the task raised one
                if result:
                    # Attempt to append the result to the CSV file
                    try:
                        pd.DataFrame([result]).to_csv(temp_path, mode='a', header=False, index=False, encoding='utf-8-sig')
                    except PermissionError:
                        print(f"PermissionError: Unable to write to {temp_path}")
                    except Exception as write_error:
                        print(f"Error writing to file: {write_error}")
            except Exception as task_error:
                print(f"Error retrieving result from future: {task_error}")

    df = pd.DataFrame(results)
    return df

Total offset we have observed is:

299970, displaying 30 per page

In [ ]:
# Example usage:
base_url = "https://www.cvh.ac.cn/spms/list.php?&offset="
offset = 170000  # Example offset, can be adjusted or iterated in a loop
data_collection_ids = scrape_data_collection_ids_for_offset(base_url, offset)

In [ ]:
data_collection_ids

['e8570e26',
 'e8570eaf',
 'e8570f38',
 'e8570fbf',
 'e8571048',
 'e85710d4',
 'e8571162',
 'e85711e9',
 'e857126f',
 'e85712f6',
 'e857137e',
 'e8571406',
 'e857148c',
 'e8571511',
 'e8571596',
 'e857161b',
 'e85716ac',
 'e857173a',
 'e85717c1',
 'e8571849',
 'e85718d1',
 'e8571959',
 'e85719e0',
 '479e8b0c',
 'e8571aef',
 'e8571b75',
 'e8571bfb',
 'e8571c81',
 'e8571d0b',
 'e8571d97']

In [ ]:
def scape_cvh(max_offset=298000, total_samples=300, sample_size=30):
    base_url = "https://www.cvh.ac.cn/spms/list.php?&offset="
    max_offset = max_offset
    # total_samples = 10000
    total_samples = total_samples
    sample_size = sample_size
    results_path = g_path

    # Load or initialize offset
    if os.path.exists(check_path):
        with open(check_path, 'r') as file:
            offset = int(file.read().strip())
    else:
        offset = max_offset

    data_collection = []

    for _ in range(total_samples // sample_size):
        # Update and wrap-around offset using modulo
        offset = (offset - sample_size) % max_offset
        print(f"Fetching data from offset: {offset}")

        # Scrape and fetch data
        data_collection_ids = scrape_data_collection_ids_for_offset(base_url, offset)
        results = fetch_collection_data_concurrently(results_path, data_collection_ids, sample_size)
        data_collection.extend(results)

        # Save the current offset
        with open(check_path, 'w') as file:
            file.write(str(offset))

    # Convert list to DataFrame
    df = pd.DataFrame(data_collection)
    df.to_csv(results_path + "/results.csv", header=True, index=False, encoding='utf-8-sig')
    print("Data collection and saving completed.")

In [ ]:
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"
check_path = "/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints/last_offset.txt"

with open(check_path, 'r') as file:
    max_offset = int(file.read().strip())

# sample size is always 30
scape_cvh(max_offset=int(max_offset), total_samples=9900, sample_size=30)

Fetching data from offset: 297040
/content/drive/MyDrive/Colab Notebooks/cs549_datasets/e96ee697.csv
Fetching data for collection: e96ee7af
.
Fetching data for collection: e96ee841
.
Fetching data for collection: e96ee9e1
.
Fetching data for collection: e96ee956
.
Fetching data for collection: e96eee3a
.
Fetching data for collection: e96eedb1
.
Fetching data for collection: e96ee722
.


  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e96eef4f
.
Fetching data for collection: e96eed21
.
Fetching data for collection: e96eeec5
.
Fetching data for collection: e96eea6c
.
Fetching data for collection: e96ef0f0
.
Fetching data for collection: e96eefda
.
Fetching data for collection: e96eeb82
.
Fetching data for collection: e96eeaf7
.
Fetching data for collection: e96ee697
.
Fetching data for collection: e96eec0c
.
Fetching data for collection: e96ef065
.Fetching data for collection: e96eec94
.
Fetching data for collection: e96ef17c
.

Fetching data for collection: e96ef299
.
Fetching data for collection: e96ee8cc
.
Fetching data for collection: e96ef321
.
Fetching data for collection: e96ef209
.
Fetching data for collection: e96ef4be
.
Fetching data for collection: e96ef3ac
.
Fetching data for collection: e96ef546
.
Fetching data for collection: e96ef435
.
Fetching data for collection: e96ef658
.
Fetching data for collection: e96ef5ce
.
Error retrieving result from future: 'str' object has no 

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e96ed647
.
Fetching data for collection: e96eda9e
.
Fetching data for collection: e96ed8f9
.
Fetching data for collection: e96edd50
.
Fetching data for collection: e96ed86f
.
Fetching data for collection: e96edbb2
.
Fetching data for collection: e96ed6d1
.
Fetching data for collection: e96eddda
.
Fetching data for collection: e96ee012
.
Fetching data for collection: e96ed75b
.
Fetching data for collection: e96ed7e5
.
Fetching data for collection: e96ede67
.
Fetching data for collection: e96edef6
.
Fetching data for collection: e96edcc5
.
Fetching data for collection: e96edb29
.
Fetching data for collection: e96ed982
.
Fetching data for collection: 47a0d766
.
Fetching data for collection: e96ee585
.
Fetching data for collection: e96eda0e
.
Fetching data for collection: e96ee127
.
Fetching data for collection: e96ee241
.
Fetching data for collection: e96ee1b6
.
Fetching data for collection: e96edc3b
.
Fetching data for collection: e96ee09d
.
Fetching data fo

...

Fetching data for collection: e9598d60
.
Fetching data for collection: e9598aad
.
Fetching data for collection: e9598a24
.
Fetching data for collection: e959899a
.
Fetching data for collection: e9598f91
.
Fetching data for collection: e9598f08
.
Fetching data for collection: e959890b
.
Fetching data for collection: e9598cd7
.
Fetching data for collection: e9598bc0
.
Fetching data for collection: e9598ded
.
Fetching data for collection: e95990a3
.
Fetching data for collection: e959901a
.
Fetching data for collection: e95991b6
.
Fetching data for collection: e9598e7e
.
Fetching data for collection: e9599244
.
Fetching data for collection: e959912c
.
Fetching data for collection: e9598c4a
.
Fetching data for collection: e959937c
.
Fetching data for collection: e9599406
.
Fetching data for collection: e9599742
.
Fetching data for collection: e959962f
.
Fetching data for collection: e95995a3
.
Fetching data for collection: e9599519
.
Fetching data for collection: e95996b8
.
Fetching data fo

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e9597a5b
.
Fetching data for collection: e959771e
.
Fetching data for collection: e9597c8c
.
Fetching data for collection: e9597da0
.
Fetching data for collection: e95977a7
.
Fetching data for collection: e959816f
.
Fetching data for collection: e9597c02
.
Fetching data for collection: e9597eb3
.
Fetching data for collection: e9597946
.
Fetching data for collection: e9597f3d
.
Fetching data for collection: e95978bc
.
Fetching data for collection: e9597ae3
.
Fetching data for collection: e95981f9
.
Fetching data for collection: e9597d16
.
Fetching data for collection: e9597e2a
.
Fetching data for collection: e9597b72
.
Fetching data for collection: e95980e6
.
Fetching data for collection: e959805c
.
Fetching data for collection: e9597832
.
Fetching data for collection: e9598426
.
Fetching data for collection: e9597fcc
.
Fetching data for collection: e95979d0
.
Fetching data for collection: e9598284
.
Fetching data for collection: e959853d
.
Fetching data fo

  0%|          | 0/30 [00:00<?, ?it/s]

Fetching data for collection: e9596746
.
Fetching data for collection: e9596ef6
.
Fetching data for collection: e9596c36
.
Fetching data for collection: e9596e6c
.
Fetching data for collection: e9596a07
.
Fetching data for collection: e9596cc6
.
Fetching data for collection: e959700c
.
Fetching data for collection: e9597468
.
Fetching data for collection: e95971b5
.
Fetching data for collection: e9596f81
.
Fetching data for collection: e9597355
.
Fetching data for collection: e9597240
.
Fetching data for collection: e95973de
.
Fetching data for collection: e95972cb
.
Fetching data for collection: e95974f0
.
Fetching data for collection: e9597124
.
Fetching data for collection: e9597096
.
Fetching data for collection: e9597606
.
Fetching data for collection: e9597579
.
Fetching data for collection: e9597695
.
Data collection and saving completed.


In [ ]:
!pwd

# Check number of samples collected

In [1]:
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"

In [4]:
import pandas as pd
import os

# Define the path to the directory containing the CSV files
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_datasets"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(g_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append to the list, skip files with errors
for file in csv_files:
    file_path = os.path.join(g_path, file)
    try:
        df = pd.read_csv(file_path)
        if not df.empty:
            dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Skipped empty or invalid file: {file}")

# Concatenate all non-empty DataFrames
if dataframes:
    combined_df = pd.concat(dataframes)
    # Replace 'missing' with NaN for easier dropping of rows
    combined_df.replace('missing', pd.NA, inplace=True)
    # Drop rows where all elements are missing (NaN)
    combined_df.dropna(how='all', inplace=True)
    # Count the number of unique rows
    unique_rows_count = combined_df.drop_duplicates().shape[0]
    print(f'Number of unique rows: {unique_rows_count}')
else:
    print("No valid CSV files found.")

Number of unique rows: 7482


In [6]:
combined_df.nunique()

Collection ID          7482
Image Link             7190
Phylum (门)                1
Order (目)                47
Family (科)              105
Genus (属)               399
Scientific Name        1413
Chinese Name           1218
Identified By           163
Date Identified         541
Collector              1202
Collection Number      5450
Collection Date        3675
Collection Location      42
Altitude                932
Habitat                2738
Phenology                 5
dtype: int64

In [10]:
g_path = "/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints"
file_path = os.path.join(g_path, 'results.csv')
combined_df.to_csv(file_path, index=False)

In [11]:
file_path

'/content/drive/MyDrive/Colab Notebooks/cs549_checkpoints/results.csv'